In [1]:
import sys
import math
sys.path.insert(1, './src')
import matplotlib
import matplotlib.pyplot as plt

import argparse
from game_ner import NERGame
from robot import RobotCNNDQN
import numpy as np
import helpers
import tensorflow as tf
import random
from tagger import CRFTagger
import pickle
import warnings; warnings.simplefilter('ignore')
print (tf.__version__)

1.14.0


In [2]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--agent', help="require a decision agent")
    parser.add_argument('--episode', help="require a maximum number of playing the game")
    parser.add_argument('--budget', help="requrie a budget for annotating")
    parser.add_argument('--train', help="training phase")
    parser.add_argument('--test', help="testing phase")
    args = parser.parse_args()
    
    global AGENT, MAX_EPISODE, BUDGET, TRAIN_LANG, TEST_LANG
    AGENT = args.agent
    MAX_EPISODE = int(args.episode)
    BUDGET = int(args.budget)
    # load the train data: source languages
    parts = args.train.split(";")
    if len(parts) % 5 != 0:
        print ("Wrong inputs of training")
        raise SystemExit
    global TRAIN_LANG_NUM
    TRAIN_LANG_NUM = len(parts) / 5
    for i in range(TRAIN_LANG_NUM):
        lang_i = i * 5
        train = parts[lang_i + 0]
        test = parts[lang_i + 1]
        dev = parts[lang_i + 2]
        emb = parts[lang_i + 3]
        tagger = parts[lang_i + 4]
        TRAIN_LANG.append((train, test, dev, emb, tagger))
    # load the test data: target languages
    parts = args.test.split(";")
    if len(parts) % 5 != 0:
        print ("Wrong inputs of testing")
        raise SystemExit
    global TEST_LANG_NUM
    TEST_LANG_NUM = len(parts) / 5
    for i in range(TEST_LANG_NUM):
        lang_i = i * 5
        train = parts[lang_i + 0]
        test = parts[lang_i + 1]
        dev = parts[lang_i + 2]
        emb = parts[lang_i + 3]
        tagger = parts[lang_i + 4]
        TEST_LANG.append((train, test, dev, emb, tagger))

def initialise_game(train_file, test_file, dev_file, emb_file, budget):
    # Load data
    print("Loading data ..")
    train_x, train_y, train_lens = helpers.load_data2labels(train_file)
    test_x, test_y, test_lens = helpers.load_data2labels(test_file)
    dev_x, dev_y, dev_lens = helpers.load_data2labels(dev_file)

    print("Processing data")
    # build vocabulary
    max_len = FLAGS.flag_values_dict()['max_seq_len']
    print ("Max document length:", max_len)
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(
        max_document_length=max_len, min_frequency=1)
    # vocab = vocab_processor.vocabulary_ # start from {"<UNK>":0}
    train_idx = np.array(list(vocab_processor.fit_transform(train_x)))
    dev_idx = np.array(list(vocab_processor.fit_transform(dev_x)))
    vocab = vocab_processor.vocabulary_
    vocab.freeze()
    test_idx = np.array(list(vocab_processor.fit_transform(test_x)))

    # build embeddings
    vocab = vocab_processor.vocabulary_
    vocab_size = FLAGS.flag_values_dict()['max_vocab_size']
    w2v = helpers.load_crosslingual_embeddings(emb_file, vocab, vocab_size)

    # prepare story
    story = [train_x, train_y, train_idx]
    print ("The length of the story ", len(train_x), " ( DEV = ", len(dev_x), " TEST = ", len(test_x), " )")
    test = [test_x, test_y, test_idx]
    dev = [dev_x, dev_y, dev_idx]
    # load game
    print("Loading game ..")
    game = NERGame(story, test, dev, max_len, w2v, budget)
    return game

In [3]:
def play_ner():
    actions = 2
    global AGENT
    robot = RobotCNNDQN(AGENT, actions)
#     if AGENT == "random":
#         robot = RobotRandom(actions)
#     elif AGENT == "DQN":
#         robot = RobotDQN(actions)
#     elif AGENT == "CNNDQN":
#         robot = RobotCNNDQN(actions)
#     else:
#         print ("** There is no robot.")
#         raise SystemExit

    global TRAIN_LANG, TRAIN_LANG_NUM, BUDGET
    for i in range(TRAIN_LANG_NUM):
        train = TRAIN_LANG[i][0]
        test = TRAIN_LANG[i][1]
        dev = TRAIN_LANG[i][2]
        emb = TRAIN_LANG[i][3]
        tagger = TRAIN_LANG[i][4]
        # initilise a NER game
        game = initialise_game(train, test, dev, emb, BUDGET)
        # initialise a decision robot
        robot.initialise(game.max_len, game.w2v)
#         robot.update_embeddings(game.w2v)
        # tagger
        model = CRFTagger(tagger)
        # play game
        episode = 1
        print(">>>>>> Playing game ..")
        while episode <= MAX_EPISODE:
            print ('>>>>>>> Current game round ', episode, 'Maximum ', MAX_EPISODE)
            observation = game.get_frame(model)
            print (observation[1])
            print (observation[3])
            action = robot.get_action(observation)
            print ('> Action', action)
            reward, observation2, terminal = game.feedback(action, model)
            print ('> Reward', reward)
            robot.update(observation, action, reward, observation2, terminal)
            if terminal == True:
                episode += 1
                print ('> Terminal <')
    return (robot, game, model)

In [4]:
def test_agent_batch(robot, game, model, budget):
    i = 0
    queried_x = []
    queried_y = []
    performance = []
    cost = []
    test_sents = helpers.data2sents(game.test_x, game.test_y)
    game.replay()
    while i < budget:
        sel_ind = game.current_frame
        # construct the observation
        observation = game.get_frame(model)
        action = robot.get_action(observation)
        if action[1] == 1:
            sentence = game.train_x[sel_ind]
            labels = game.train_y[sel_ind]
            queried_x.append(sentence)
            queried_y.append(labels)
            i += 1
            train_sents = helpers.data2sents(queried_x, queried_y)
            model.train(train_sents)
            performance.append(model.test(test_sents))
            cost.append(i)
        game.current_frame += 1
    print ('***COST', cost)
    print ("***TEST", performance)
    return (cost, performance)

def test_agent_online(robot, game, model, budget):
    # to address game -> we have a new game here
    i = 0
    queried_x = []
    queried_y = []
    performance = []
    test_sents = helpers.data2sents(game.test_x, game.test_y)
    game.reboot()
    while i < budget:
        sel_ind = game.current_frame
        # construct the observation
        observation = game.get_frame(model)
        action = robot.get_action(observation)
        if action[1] == 1:
            sentence = game.train_x[sel_ind]
            labels = game.train_y[sel_ind]
            queried_x.append(sentence)
            queried_y.append(labels)
            i += 1
            train_sents = helpers.data2sents(queried_x, queried_y)
            model.train(train_sents)
            performance.append(model.test(test_sents))

        reward, observation2, terminal = game.feedback(action, model)  # game
        robot.update(observation, action, reward, observation2, terminal)
    # train a crf and evaluate it
    train_sents = helpers.data2sents(queried_x, queried_y)
    model.train(train_sents)
    performance.append(model.test(test_sents))
    print ("***TEST", performance)

def test(robot):
    global TEST_LANG, TEST_LANG_NUM, BUDGET
    for i in range(TEST_LANG_NUM):
        train = TEST_LANG[i][0]
        test = TEST_LANG[i][1]
        dev = TEST_LANG[i][2]
        emb = TEST_LANG[i][3]
        tagger = TEST_LANG[i][4]
        game2 = initialise_game(train, test, dev, emb, BUDGET)
        robot.update_embeddings(game2.w2v)
        model = CRFTagger(tagger)
        test_agent_batch(robot, game2, model, BUDGET)
        test_agent_online(robot, game2, model, BUDGET)

In [5]:
flags = tf.flags
FLAGS = flags.FLAGS
flags.DEFINE_integer("max_seq_len", 120, "sequence")
flags.DEFINE_integer("max_vocab_size", 20000, "vocabulary")
print("\nParameters:")
for attr, value in FLAGS.flag_values_dict().items():
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
LOGTOSTDERR=False
ALSOLOGTOSTDERR=False
LOG_DIR=
V=-1
VERBOSITY=-1
STDERRTHRESHOLD=fatal
SHOWPREFIXFORINFO=True
RUN_WITH_PDB=False
PDB_POST_MORTEM=False
RUN_WITH_PROFILING=False
PROFILE_FILE=None
USE_CPROFILE_FOR_PROFILING=True
ONLY_CHECK_ARGS=False
OP_CONVERSION_FALLBACK_TO_WHILE_LOOP=False
TEST_RANDOM_SEED=301
TEST_SRCDIR=
TEST_TMPDIR=/tmp/absl_testing
TEST_RANDOMIZE_ORDERING_SEED=None
XML_OUTPUT_FILE=
MAX_SEQ_LEN=120
MAX_VOCAB_SIZE=20000



In [6]:
# parse_args()
global AGENT, MAX_EPISODE, BUDGET, TRAIN_LANG, TEST_LANG, TRAIN_LANG_NUM, TEST_LANG_NUM
SOURCE = 'conll'
AGENT = 'CNNDQN'
MAX_EPISODE = 1
BUDGET = 5

DATAPATH = './datasets/{}'.format(SOURCE)
train_f  = DATAPATH + '.train'
test_f   = DATAPATH + '.test'
dev_f    = DATAPATH + '.dev'
emb_f    = DATAPATH + 'NUM.kv'
tagger_f = DATAPATH + 'model.saved'

TRAIN_LANG = []
TEST_LANG = []
TRAIN_LANG_NUM = 1
TRAIN_LANG.append((train_f, test_f, dev_f, emb_f, tagger_f))
TEST_LANG_NUM = 1
TEST_LANG.append((train_f, test_f, dev_f, emb_f, tagger_f))

In [7]:
# play games for training a robot
robot, game, model = play_ner()
cost_list, acc_list = test_agent_batch(robot, game, model, BUDGET)
acc_valid_list = acc_list

W0930 07:54:05.882400 140714725021504 deprecation_wrapper.py:119] From ./src/robot.py:202: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 07:54:05.901150 140714725021504 deprecation_wrapper.py:119] From ./src/robot.py:212: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0930 07:54:05.910346 140714725021504 deprecation_wrapper.py:119] From ./src/robot.py:224: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0930 07:54:05.919106 140714725021504 deprecation_wrapper.py:119] From ./src/robot.py:237: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0930 07:54:05.939310 140714725021504 deprecation.py:506] From ./src/robot.py:250: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate

Creating a robot: CNN-DQN


W0930 07:54:06.225312 140714725021504 deprecation_wrapper.py:119] From ./src/robot.py:97: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0930 07:54:06.791177 140714725021504 deprecation_wrapper.py:119] From ./src/robot.py:99: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

W0930 07:54:06.876793 140714725021504 deprecation_wrapper.py:119] From ./src/robot.py:38: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



Loading data ..
Processing data
Max document length: 120


W0930 07:54:08.852131 140714725021504 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0930 07:54:08.853000 140714725021504 deprecation.py:323] From <ipython-input-2-9d0fe32ac77c>:57: VocabularyProcessor.__init__ (from tensorflow.contrib.learn.python.learn.preprocessing.text) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.
W0930 07:54:08.853412 140714725021504 deprecation.py:323] From /zf18/ll5fy/miniconda3/envs/ll5fy36/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/preprocessing/text.py:154: CategoricalVocabulary.__init__ (from tensorflow.contrib.learn.python.

Vocabulary 1339 Embedding size 200
The length of the story  615  ( DEV =  200  TEST =  200  )
Loading game ..
Initilizing the game:
Story: length =  615
CRF Tagger
>>>>>> Playing game ..
>>>>>>> Current game round  1 Maximum  1
[0.8608480576077683]
[5]
DQN is smart.


ValueError: setting an array element with a sequence.

In [ ]:
with open("./results/{}-{}.bin".format(SOURCE, MAX_EPISODE), "wb") as result:
    pickle.dump((cost_list, acc_list, acc_valid_list), result)

In [ ]:
with open("./results/conll-20.bin", "rb") as in_file:
    (cost_1, acc_1, acc_valid_1) = pickle.load(in_file)
print (cost_1)
print (acc_1)

## plot result

In [ ]:
# sim beta
with open("./results/CNNDQN-conll-20.bin", "rb") as in_file:
    (cost_1, all_acc_1, acc_valid_1) = pickle.load(in_file)
#     (cost_1, all_acc_1, acc_valid_1) = pickle.load(in_file)
       
with open("./results/CNNDQN1-conll-20.bin", "rb") as in_file:
    (cost_2, all_acc_2, acc_valid_2) = pickle.load(in_file)

with open("./results/CNNDQN2-conll-20.bin", "rb") as in_file:
    (cost_3, all_acc_3, acc_valid_3) = pickle.load(in_file)

with open("./results/CNNDQN3-conll-20.bin", "rb") as in_file:
    (cost_4, all_acc_4, acc_valid_4) = pickle.load(in_file)
    
with open("../active-RL/results/conll15_200_75budget_fully_none.bin", "rb") as in_file:
    (cost_5, _, _, all_acc_5, acc_valid_5) = pickle.load(in_file)

# with open("./results/CNNDQN_2-conll-20.bin", "rb") as in_file:
#     (cost_6, all_acc_6, acc_valid_6) = pickle.load(in_file)

# with open("./results/CNNDQN_3-conll-20.bin", "rb") as in_file:
#     (cost_7, all_acc_7, acc_valid_7) = pickle.load(in_file)

plt.plot(cost_1[15:], all_acc_1[15:],
         cost_2[15:], all_acc_2[15:],
         cost_3[15:], all_acc_3[15:],
         cost_4[15:], all_acc_4[15:],
         cost_5, all_acc_5,)
#          cost_6[15:], all_acc_6[15:],
#          cost_7[15:], all_acc_7[15:])
plt.legend(['all', '1', '2', '3', 'TE' ,'-2', '-3'], loc='upper left', fancybox=True, fontsize = 9)
# plt.xlim(200, 600)

plt.title('CoNLL dataset')
plt.xlabel('Number of training sequences')
plt.ylabel('Prediction accuracy')
plt.savefig('./results/sod.png', bbox_inches='tight')
